# [CNN]

This notebook is given just for curiosity. This method wasn't used because of a high computation time and poor results

_______________________

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  

Loading data

In [2]:
train_df = pd.read_csv("/kaggle/input/2el1730-img-cnn-preprocess/train_df_preprocessed5.csv")
test_df = pd.read_csv("/kaggle/input/2el1730-img-cnn-preprocess/test_df_preprocessed5.csv")

In [3]:
train_df.columns

Index(['urban_type', 'geography_type', 'change_type', 'img_red_mean_date0_old',
       'img_green_mean_date0_old', 'img_blue_mean_date0_old',
       'img_red_std_date0_old', 'img_green_std_date0_old',
       'img_blue_std_date0_old', 'img_red_mean_date1_old',
       ...
       'geometry_solidity', 'geometry_diameter',
       'geometry_inscribed_circle_radius', 'geometry_lat', 'geometry_long',
       'change_status_date_4-0xDate', 'change_status_date_1-0xDate',
       'change_status_date_2-1xDate', 'change_status_date_3-2xDate',
       'change_status_date_4-3xDate'],
      dtype='object', length=278)

In [4]:
for column, typ in zip(train_df.columns, train_df.dtypes):
    if typ in ['object', 'datetime64[ns]', 'geometry']:
        continue
    print("'", end="")
    print(column, end="', ")

'change_type', 'img_red_mean_date0_old', 'img_green_mean_date0_old', 'img_blue_mean_date0_old', 'img_red_std_date0_old', 'img_green_std_date0_old', 'img_blue_std_date0_old', 'img_red_mean_date1_old', 'img_green_mean_date1_old', 'img_blue_mean_date1_old', 'img_red_std_date1_old', 'img_green_std_date1_old', 'img_blue_std_date1_old', 'img_red_mean_date2_old', 'img_green_mean_date2_old', 'img_blue_mean_date2_old', 'img_red_std_date2_old', 'img_green_std_date2_old', 'img_blue_std_date2_old', 'img_red_mean_date3_old', 'img_green_mean_date3_old', 'img_blue_mean_date3_old', 'img_red_std_date3_old', 'img_green_std_date3_old', 'img_blue_std_date3_old', 'img_red_mean_date4_old', 'img_green_mean_date4_old', 'img_blue_mean_date4_old', 'img_red_std_date4_old', 'img_green_std_date4_old', 'img_blue_std_date4_old', 'index', 'urban_type_N,A', 'urban_type_UrbanSlum', 'urban_type_DenseUrban', 'urban_type_SparseUrban', 'urban_type_Industrial', 'urban_type_Rural', 'geography_type_GrassLand', 'geography_type

In [5]:
feature_names=['urban_type_N,A', 'urban_type_UrbanSlum', 'urban_type_DenseUrban', 'urban_type_SparseUrban', 'urban_type_Industrial', 'urban_type_Rural', 
                'geography_type_GrassLand', 'geography_type_BarrenLand', 'geography_type_N,A', 'geography_type_Hills', 'geography_type_River', 'geography_type_Coastal', 'geography_type_SparseForest', 'geography_type_Farms', 'geography_type_Desert', 'geography_type_DenseForest', 'geography_type_Lakes', 'geography_type_Snow', 
                'img_red_mean_date0', 'img_red_mean_date1', 'img_red_mean_date2', 'img_red_mean_date3', 'img_red_mean_date4', 'img_green_mean_date0', 'img_green_mean_date1', 'img_green_mean_date2', 'img_green_mean_date3', 'img_green_mean_date4', 'img_blue_mean_date0', 'img_blue_mean_date1', 'img_blue_mean_date2', 'img_blue_mean_date3', 'img_blue_mean_date4', 'img_red_std_date0', 'img_red_std_date1', 'img_red_std_date2', 'img_red_std_date3', 'img_red_std_date4', 'img_green_std_date0', 'img_green_std_date1', 'img_green_std_date2', 'img_green_std_date3', 'img_green_std_date4', 'img_blue_std_date0', 'img_blue_std_date1', 'img_blue_std_date2', 'img_blue_std_date3', 'img_blue_std_date4', 
                'change_status_date0', 'change_status_date1', 'change_status_date2', 'change_status_date3', 'change_status_date4', 
                'change_status_date0_Construction Done', 'change_status_date0_Construction Midway', 'change_status_date0_Construction Started', 'change_status_date0_Excavation', 'change_status_date0_Greenland', 'change_status_date0_Land Cleared', 'change_status_date0_Materials Dumped', 'change_status_date0_Materials Introduced', 'change_status_date0_Operational', 'change_status_date0_Prior Construction', 'change_status_date1_Construction Done', 'change_status_date1_Construction Midway', 'change_status_date1_Construction Started', 'change_status_date1_Excavation', 'change_status_date1_Greenland', 'change_status_date1_Land Cleared', 'change_status_date1_Materials Dumped', 'change_status_date1_Materials Introduced', 'change_status_date1_Operational', 'change_status_date1_Prior Construction', 'change_status_date2_Construction Done', 'change_status_date2_Construction Midway', 'change_status_date2_Construction Started', 'change_status_date2_Excavation', 'change_status_date2_Greenland', 'change_status_date2_Land Cleared', 'change_status_date2_Materials Dumped', 'change_status_date2_Materials Introduced', 'change_status_date2_Operational', 'change_status_date2_Prior Construction', 'change_status_date3_Construction Done', 'change_status_date3_Construction Midway', 'change_status_date3_Construction Started', 'change_status_date3_Excavation', 'change_status_date3_Greenland', 'change_status_date3_Land Cleared', 'change_status_date3_Materials Dumped', 'change_status_date3_Materials Introduced', 'change_status_date3_Operational', 'change_status_date3_Prior Construction', 'change_status_date4_Construction Done', 'change_status_date4_Construction Midway', 'change_status_date4_Construction Started', 'change_status_date4_Excavation', 'change_status_date4_Greenland', 'change_status_date4_Land Cleared', 'change_status_date4_Materials Dumped', 'change_status_date4_Materials Introduced', 'change_status_date4_Operational', 'change_status_date4_Prior Construction', 
                'change_status_date_4-0', 'change_status_date_1-0', 'change_status_date_2-1', 'change_status_date_3-2', 'change_status_date_4-3', 
                'img_red_mean_date_4-0', 'img_red_mean_date_1-0', 'img_red_mean_date_2-1', 'img_red_mean_date_3-2', 'img_red_mean_date_4-3', 'img_green_mean_date_4-0', 'img_green_mean_date_1-0', 'img_green_mean_date_2-1', 'img_green_mean_date_3-2', 'img_green_mean_date_4-3', 'img_blue_mean_date_4-0', 'img_blue_mean_date_1-0', 'img_blue_mean_date_2-1', 'img_blue_mean_date_3-2', 'img_blue_mean_date_4-3', 'img_red_std_date_4-0', 'img_red_std_date_1-0', 'img_red_std_date_2-1', 'img_red_std_date_3-2', 'img_red_std_date_4-3', 'img_green_std_date_4-0', 'img_green_std_date_1-0', 'img_green_std_date_2-1', 'img_green_std_date_3-2', 'img_green_std_date_4-3', 'img_blue_std_date_4-0', 'img_blue_std_date_1-0', 'img_blue_std_date_2-1', 'img_blue_std_date_3-2', 'img_blue_std_date_4-3', 
                'date_1-0', 'date_2-1', 'date_3-2', 'date_4-3', 'date_4-0', 'date_0-0min', 
                'img_red_mean_date_4-0xDate', 'img_red_mean_date_1-0xDate', 'img_red_mean_date_2-1xDate', 'img_red_mean_date_3-2xDate', 'img_red_mean_date_4-3xDate', 'img_green_mean_date_4-0xDate', 'img_green_mean_date_1-0xDate', 'img_green_mean_date_2-1xDate', 'img_green_mean_date_3-2xDate', 'img_green_mean_date_4-3xDate', 'img_blue_mean_date_4-0xDate', 'img_blue_mean_date_1-0xDate', 'img_blue_mean_date_2-1xDate', 'img_blue_mean_date_3-2xDate', 'img_blue_mean_date_4-3xDate', 'img_red_std_date_4-0xDate', 'img_red_std_date_1-0xDate', 'img_red_std_date_2-1xDate', 'img_red_std_date_3-2xDate', 'img_red_std_date_4-3xDate', 'img_green_std_date_4-0xDate', 'img_green_std_date_1-0xDate', 'img_green_std_date_2-1xDate', 'img_green_std_date_3-2xDate', 'img_green_std_date_4-3xDate', 'img_blue_std_date_4-0xDate', 'img_blue_std_date_1-0xDate', 'img_blue_std_date_2-1xDate', 'img_blue_std_date_3-2xDate', 'img_blue_std_date_4-3xDate', 
                'change_status_date_4-0xDate', 'change_status_date_1-0xDate', 'change_status_date_2-1xDate', 'change_status_date_3-2xDate', 'change_status_date_4-3xDate',
                #'geo_cluster', 
                'geo_cc_AE', 'geo_cc_AO', 'geo_cc_AU', 'geo_cc_BD', 'geo_cc_BT', 'geo_cc_CA', 'geo_cc_CD', 'geo_cc_CI', 'geo_cc_CN', 'geo_cc_DZ', 'geo_cc_EG', 'geo_cc_ET', 'geo_cc_FR', 'geo_cc_IN', 
               #'geo_cc_IT', 'geo_cc_JP',
               'geo_cc_KE', 'geo_cc_LA', 'geo_cc_MN', 'geo_cc_MU', 'geo_cc_MX', 'geo_cc_QA', 'geo_cc_RU', 'geo_cc_SA', 'geo_cc_SE', 'geo_cc_TZ', 'geo_cc_US', 'geo_cc_VN', 'geo_cc_ZA', 
                'geometry_area', 'geometry_perimeter', 'geometry_vertices', 'geometry_sides', 'geometry_bboxwidth', 'geometry_bboxheight', 'geometry_bboxarea', 'geometry_bboxperimeter', 'geometry_compactness', 'geometry_aspectratio', 'geometry_convexity', 'geometry_solidity', 'geometry_diameter', 'geometry_inscribed_circle_radius', 'geometry_lat', 'geometry_long'
               ]

In [6]:
X_train = train_df[feature_names].to_numpy()
X_train_index = train_df[['index']].to_numpy()

Y_train = train_df['change_type'].to_numpy()

X_test = test_df[feature_names].to_numpy()
X_test_index = test_df[['index']].to_numpy()

X = np.concatenate([X_train, X_test], axis=0)

# Data normalization

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Data imputation

In [8]:
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp_mean.fit(X)
X_test = imp_mean.transform(X_test)

# Adding back indexes

In [9]:
X_train = np.concatenate([X_train, X_train_index], axis=1).astype(np.float32)
X_test = np.concatenate([X_test, X_test_index], axis=1).astype(np.float32)
X = np.concatenate([X_train, X_test], axis=0).astype(np.float32)

In [10]:
X_train.shape

(292758, 223)

# Database

https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
https://stackoverflow.com/questions/73266661/multiple-input-model-with-pytorch-input-and-output-features

In [11]:
import os
import torch
import pandas as pd
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

class CustomDataset(Dataset):

    def __init__(self, data, labels, img_dir, preprocess=False):
        """
        Arguments:
            - data : N * M+1 numpy array, with N the number of samples, and the last column as the index for the photo
        """
        self.data = data
        self.labels = labels
        self.img_dir = img_dir
        
        if preprocess:
            self.preprocess = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
        else:
            self.preprocess = transforms.ToTensor()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = f"{self.img_dir}/{int(self.data[idx, -1])}.png"
        
        image = Image.open(img_name)
        #image = io.imread(img_name, as_gray=False)
        #image = np.expand_dims(image, axis=0)
        #image = image.transpose((2, 0, 1))
        features = self.data[idx][:-1]
        
        label = self.labels[idx]
        
        image = self.preprocess(image)
        sample = {'image': image, 'features': torch.from_numpy(features), 'label': self.labels[idx]}

        return sample

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torch.nn.functional as F


In [13]:

class CNN_VGG(nn.Module):
    def __init__(self):
        
        super(CNN_VGG, self).__init__()
        
        FEATURES_SIZE = X.shape[1]-1
        NB_OUT_IMG = 32
        NB_CLASSES = 6
        
        self.image_features_v2 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 2, stride = 2),
        
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            
            # Flatten the output for fully connected layers
            nn.Flatten(),
            nn.LazyLinear(128),
            nn.ReLU(),
            nn.LazyLinear(32),
        )
        
        self.combined_features = nn.Sequential(
            # change this input nodes
            nn.ReLU(),
            nn.Linear(32,NB_CLASSES)
        )
    
    def extract_IMGfeatures(self, image, features):
        a = self.image_features_v2(image)
        return a
        
    def forward(self, image, features):
        a = self.image_features_v2(image)
        #x = a.view(a.size(0), -1)
        x = self.combined_features(a)
        #x = torch.sigmoid(x)
        return x


In [14]:
from torchvision.models import resnet50, ResNet50_Weights

test = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
newmodel = torch.nn.Sequential(*(list(test.children())[:-1]))

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 67.6MB/s]


In [15]:
from torchvision.models import resnet50, ResNet50_Weights

class MixedNetworkResnet(nn.Module):
    def __init__(self):
        super(MixedNetworkResnet, self).__init__()
        
        #image_modules = list(models.resnet50().children())[:-1]
        #self.image_features = nn.Sequential(*image_modules)
        
        IMG_SIZE = 64
        FEATURES_SIZE = X.shape[1]-1
        NB_OUT_IMG = 32
        NB_OUT_FEATURES = 64
        NB_CLASSES = 6
    
        resnet = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        self.image_features = torch.nn.Sequential(
            *(list(resnet.children())[:-1]),
            nn.LazyLinear(NB_OUT_IMG)
        )
        
        self.features = nn.Sequential(
            nn.Linear(in_features=FEATURES_SIZE, out_features=128), 
            nn.ReLU(), 
            nn.LazyLinear(128),
            nn.ReLU(),
            nn.LazyLinear(NB_OUT_FEATURES), 
        )
        
        self.combined_features = nn.Sequential(
            # change this input nodes
            nn.LazyLinear(64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64,NB_CLASSES)
        )
    
    def extract_NOIMGfeatures(self, image, features):
        b = self.features(features)
        return b
    
    def extract_IMGfeatures(self, image, features):
        a = self.image_features(image)
        return a
    
    def forward(self, image, features):
        a = self.image_features(image)
        b = self.features(features)
        x = torch.cat((a.view(a.size(0), -1), b.view(b.size(0), -1)), dim=1)
        x = self.combined_features(x)
        #x = torch.sigmoid(x)
        return x


In [16]:

class MixedNetwork(nn.Module):
    def __init__(self):
        super(MixedNetwork, self).__init__()
        
        #image_modules = list(models.resnet50().children())[:-1]
        #self.image_features = nn.Sequential(*image_modules)
        
        IMG_SIZE = 64
        FEATURES_SIZE = X.shape[1]-1
        NB_OUT_IMG = 32
        NB_OUT_FEATURES = 64
        NB_CLASSES = 6
    
        self.image_features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 2, stride = 2),
        
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            
            # Flatten the output for fully connected layers
            nn.Flatten(),

            nn.LazyLinear(128),
            nn.ReLU(),
            nn.LazyLinear(NB_OUT_IMG)
        )
        
        self.features = nn.Sequential(
            nn.Linear(in_features=FEATURES_SIZE, out_features=128), 
            nn.ReLU(), 
            nn.LazyLinear(128),
            nn.ReLU(),
            nn.LazyLinear(NB_OUT_FEATURES), 
        )
        
        self.combined_features = nn.Sequential(
            # change this input nodes
            nn.Linear(NB_OUT_FEATURES + NB_OUT_IMG, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64,NB_CLASSES)
        )
    
    def extract_NOIMGfeatures(self, image, features):
        b = self.features(features)
        return b
    
    def extract_IMGfeatures(self, image, features):
        a = self.image_features(image)
        return a
    
    def forward(self, image, features):
        a = self.image_features(image)
        b = self.features(features)
        x = torch.cat((a.view(a.size(0), -1), b.view(b.size(0), -1)), dim=1)
        x = self.combined_features(x)
        #x = torch.sigmoid(x)
        return x


In [17]:

model = MixedNetworkResnet()
model

MixedNetworkResnet(
  (image_features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
        

# Data

In [18]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, test_size=0.2, random_state=42, stratify=Y_train)

In [19]:
reduce = False
if reduce:
    x_train = x_train[:1000]
    y_train = y_train[:1000]
    x_valid = x_valid[:1000]
    y_valid = y_valid[:1000]
    X_test = X_test[:1000]


In [20]:
real_train_dataset = CustomDataset(X_train, Y_train, "/kaggle/input/2el1730-images/2EL1730-CS/img/train", preprocess=True)

In [21]:
train_dataset = CustomDataset(x_train, y_train, "/kaggle/input/2el1730-images/2EL1730-CS/img/train", preprocess=True)
valid_dataset = CustomDataset(x_valid, y_valid, "/kaggle/input/2el1730-images/2EL1730-CS/img/train", preprocess=True)
test_dataset = CustomDataset(X_test, [0 for i in range(len(X_test))], "/kaggle/input/2el1730-images/2EL1730-CS/img/test", preprocess=True)

In [22]:
x_valid[1]

array([-3.71883899e-01, -1.25037774e-01,  1.45390403e+00, -6.05429053e-01,
       -6.03712618e-01, -2.75929540e-01, -8.07117283e-01, -4.51800019e-01,
       -1.54391631e-01, -5.98477870e-02,  2.65110564e+00, -1.49845868e-01,
        6.19473934e-01,  1.91770494e+00, -1.46011263e-01, -5.85553765e-01,
       -4.34881240e-01, -9.33352858e-03,  2.89307147e-01, -8.21207821e-01,
        1.12322795e+00, -7.93818891e-01,  6.84644520e-01,  1.59421682e-01,
       -5.31763852e-01,  1.23603487e+00, -7.41214991e-01,  2.29958445e-01,
        1.65763676e-01, -6.55862093e-01,  3.98914993e-01, -8.55486751e-01,
        3.84983164e-03, -6.89185560e-01,  7.04278231e-01, -1.54540193e+00,
       -2.87559628e-01,  1.81166828e-01, -5.90023339e-01,  5.62671185e-01,
       -1.49417174e+00, -3.26084375e-01,  2.15538517e-01, -5.28402984e-01,
        7.25789011e-01, -1.41695523e+00, -3.70109975e-01,  3.08921695e-01,
        1.42485142e+00,  1.33365035e+00, -1.01127160e+00, -1.33916688e+00,
       -1.78796375e+00, -

# Weighted random sampler

In [23]:
class_counts = np.unique(Y_train, return_counts=True)[1]

In [24]:
class_counts

array([ 31156,  14190, 146453,  99500,   1310,    149])

In [25]:
np.unique(y_train, return_counts=True)[1]

array([ 24925,  11352, 117162,  79600,   1048,    119])

In [26]:
np.unique(y_valid, return_counts=True)[1]

array([ 6231,  2838, 29291, 19900,   262,    30])

In [27]:
class_weights = [1/class_counts[i] for i in range(6)]

In [28]:
"""
from torch.utils.data import WeightedRandomSampler

sampler_train = WeightedRandomSampler(weights=[class_weights[i] for i in y_train], num_samples=len(train_dataset), replacement=True)
sampler_valid = WeightedRandomSampler(weights=[class_weights[i] for i in y_valid], num_samples=len(valid_dataset), replacement=True)
"""

'\nfrom torch.utils.data import WeightedRandomSampler\n\nsampler_train = WeightedRandomSampler(weights=[class_weights[i] for i in y_train], num_samples=len(train_dataset), replacement=True)\nsampler_valid = WeightedRandomSampler(weights=[class_weights[i] for i in y_valid], num_samples=len(valid_dataset), replacement=True)\n'

In [29]:
class_weights = np.array(class_weights)
class_weights*100000

array([  3.20965464,   7.04721635,   0.68281292,   1.00502513,
        76.33587786, 671.1409396 ])

In [30]:
BATCH_SIZE = 64
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=2, shuffle=True)#sampler=sampler_train)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, num_workers=2, shuffle=True)#sampler=sampler_valid)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [31]:
inputs = train_dataset[0]
images = inputs["image"]
features = inputs["features"]
labels = inputs["label"]
images.shape

torch.Size([3, 224, 224])

In [32]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor([3.2, 7, 0.7, 1, 70, 300]).to(device))
optimizer = optim.Adam(model.parameters(), lr=0.001)


model.to(device)
print("Train dataloader:",len(train_dataloader))
print("Valid dataloader:", len(valid_dataloader))

for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    
    all_labels = []
    all_predictions = []
    
    print("===== EPOCH", (epoch+1), "====")
    i = 0
    for inputs in train_dataloader:
        if i%100 == 0:
            print(i, end=" ")
        i+=1
        images = inputs["image"].to(device)
        features = inputs["features"].to(device)
        labels = inputs["label"].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images, features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()
        
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
        
        
    average_loss = total_loss / len(train_dataloader)
    accuracy = total_correct / total_samples # accuracy_score(all_labels, all_predictions)
    f1_score_micro = f1_score(all_labels, all_predictions, average='micro')
    f1_score_weighted = f1_score(all_labels, all_predictions, average='weighted')

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}")
    print(f"F1 Score Micro: {f1_score_micro:.4f}, F1 Score Weighted: {f1_score_weighted:.4f}")
    
    train_conf_matrix = confusion_matrix(all_labels, all_predictions)
    print(train_conf_matrix)
    
    # Validation phase
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        print("Validation...")
        i = 0
        for inputs in valid_dataloader:
            if i%100 == 0:
                print(i, end=" ")
            i += 1
            images = inputs["image"].to(device)
            features = inputs["features"].to(device)
            labels = inputs["label"].to(device)

            outputs = model(images, features)
            _, predicted = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    val_accuracy = accuracy_score(all_labels, all_predictions)
    val_f1_score_micro = f1_score(all_labels, all_predictions, average='micro')
    val_f1_score_weighted = f1_score(all_labels, all_predictions, average='weighted')

    print(f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score Micro: {val_f1_score_micro:.4f}, Validation F1 Score Weighted: {val_f1_score_weighted:.4f}")

    valid_conf_matrix = confusion_matrix(all_labels, all_predictions)
    print(valid_conf_matrix)
    
print("Training complete!")

Train dataloader: 3660
Valid dataloader: 915
===== EPOCH 1 ====
0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 Epoch 1/10, Loss: 1.2035, Accuracy: 0.4868
F1 Score Micro: 0.4868, F1 Score Weighted: 0.4962
[[22482   318  1034  1021    60    10]
 [  170  8425   955  1621   176     5]
 [ 7412 19055 59868 28905  1892    30]
 [ 5047 19876 30177 23215  1269    16]
 [   76   180   501   268    23     0]
 [   12    26    55    24     2     0]]
Validation...
0 100 200 300 400 500 600 700 800 900 Validation Accuracy: 0.6299, Validation F1 Score Micro: 0.6299, Validation F1 Score Weighted: 0.5745
[[ 5844    32   312    34     1     8]
 [   26  1547   332   932     0     1]
 [ 1275   202 25884  1895     0    35]
 [  680   664 14912  3609     0    35]
 [   14     8   228    12     0     0]
 [    6     2    15     7     0     0]]
===== EPOCH 2 ====
0 100 200 300 400 500 600 70

In [33]:

# Validation phase

model.eval()
all_predictions = []

with torch.no_grad():
    for inputs in test_dataloader:

        images = inputs["image"].to(device)
        features = inputs["features"].to(device)

        outputs = model(images, features)
        _, predicted = torch.max(outputs, 1)

        all_predictions.extend(predicted.cpu().numpy())


pred_df = pd.DataFrame(all_predictions, columns=['change_type'])
pred_df.to_csv("cnn_sample_submission.csv", index=True, index_label='Id')


In [34]:

# Validation phase
model.eval()
all_features_train = []
all_features_test = []

with torch.no_grad():
    print("Computing IMG features for training", len(real_train_dataset))
    for i in range(len(real_train_dataset)):
        if i%1000 == 0:
            print(i)

        inputs = real_train_dataset[i]
        images = inputs["image"].to(device)
        features = inputs["features"].to(device)

        outputs = model.extract_IMGfeatures(images.unsqueeze(0), features).cpu().numpy()
        
        all_features_train.extend(outputs)
        
        #if i > 10:
        #    break
    
    
    print("Computing IMG features for test", len(test_dataset))
    for i in range(len(test_dataset)):
        if i%1000 == 0:
            print(i)

        inputs = test_dataset[i]
        images = inputs["image"].to(device)
        features = inputs["features"].to(device)
        
        outputs = model.extract_IMGfeatures(images.unsqueeze(0), features).cpu().numpy()
        
        all_features_test.extend(outputs)
        
        #print(outputs.shape)
        #if i > 10:
        #    break

all_features_train = np.array(all_features_train)
all_features_test = np.array(all_features_test)

#print("TRAIN")
#print(all_features_train)

#print("TEST")
#print(all_features_test)

np.save("X_train_IMG.npy", all_features_train)
np.save("X_test_IMG.npy", all_features_test)
    

Computing IMG features for training 292758
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000


In [ ]:

# Validation phase
model.eval()
all_features_train = []
all_features_test = []

with torch.no_grad():
    print("Computing NOIMG features for training", len(real_train_dataset))
    for i in range(len(real_train_dataset)):
        if i%1000 == 0:
            print(i)

        inputs = real_train_dataset[i]
        images = inputs["image"].to(device)
        features = inputs["features"].to(device)

        outputs = model.extract_NOIMGfeatures(images.unsqueeze(0), features).cpu().numpy()
        
        all_features_train.extend(outputs)
        
        #if i > 10:
        #    break
    
    
    print("Computing IMG features for test", len(test_dataset))
    for i in range(len(test_dataset)):
        if i%1000 == 0:
            print(i)

        inputs = test_dataset[i]
        images = inputs["image"].to(device)
        features = inputs["features"].to(device)
        
        outputs = model.extract_NOIMGfeatures(images.unsqueeze(0), features).cpu().numpy()
        
        all_features_test.extend(outputs)
        
        #print(outputs.shape)
        #if i > 10:
        #    break

all_features_train = np.array(all_features_train)
all_features_test = np.array(all_features_test)

#print("TRAIN")
#print(all_features_train)

#print("TEST")
#print(all_features_test)

np.save("X_train_NOIMG.npy", all_features_train)
np.save("X_test_NOIMG.npy", all_features_test)
    

In [ ]:
all_features_train.shape

In [ ]:
all_features_test.shape